In [5]:
import slippi
from slippi import Game
import pickle
import pandas as pd
import glob
import numpy as np
from tqdm import tqdm 
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

pd.set_option('display.max_columns', None)

In [2]:
path = 'replays/Fight-Pitt-9/'
fp9_files = glob.glob(path + '/**/*.slp', recursive=True)

path = 'replays/Gang-Steals/'
gs_files = glob.glob(path + '/**/*.slp', recursive=True)

In [3]:
len(fp9_files)

1150

In [4]:
len(gs_files)

3561

In [9]:
Game(fp9_files[0])

Game(
    end=End(
        lras_initiator=None,
        method=3:CONCLUSIVE),
    frames=[...](8175),
    metadata=Metadata(
        console_name=Station 33,
        date=2019-03-09 00:19:43+00:00,
        duration=8175,
        platform='nintendont':NINTENDONT,
        players=(None, None, None, None)),
    start=Start(
        is_frozen_ps=None,
        is_pal=False,
        is_teams=False,
        players=(
            Player(
                character=2:FOX,
                costume=0,
                stocks=4,
                tag=,
                team=None,
                type=0:HUMAN,
                ucf=UCF(
                    dash_back=1:UCF,
                    shield_drop=1:UCF)),
            None,
            None,
            Player(
                character=9:MARTH,
                costume=1,
                stocks=4,
                tag=。,
                team=None,
                type=0:HUMAN,
                ucf=UCF(
                    dash_back=1:UCF,
            

In [5]:
Game(fp9_files[1]).frames[-1]

Frame(
    end=None,
    index=9578,
    items=(),
    ports=(
        Port(
            follower=None,
            leader=Data(
                post=Post(
                    airborne=None,
                    character=1:FOX,
                    combo_count=1,
                    damage=126.97,
                    direction=-1:LEFT,
                    flags=None,
                    ground=None,
                    hit_stun=None,
                    jumps=None,
                    l_cancel=None,
                    last_attack_landed=21:DOWN_SPECIAL,
                    last_hit_by=None,
                    position=(70.32, -14.40),
                    shield=60.00,
                    state=253:CLIFF_WAIT,
                    state_age=23.00,
                    stocks=1),
                pre=Pre(
                    buttons=Buttons(
                        logical=0b0:NONE,
                        physical=0b0:NONE),
                    cstick=(0.00, 0.00),
                    dam

In [12]:
def get_ports(game):
    player_tup = game.start.players
    ports = tuple([i for i in range(4) if player_tup[i] is not None])
    return ports

def get_characters(game, port1, port2):
    player_tup = game.start.players
    chars = (player_tup[port1].character.name,player_tup[port2].character.name )
    return chars
    
def get_stage(game):
    return game.start.stage.name
    
def get_winner(game, port1, port2):
    """
    Returns if player with lower port is winner
    """
    lras = game.end.lras_initiator
    if lras is not None:
        if lras == port1:
            return 0
        else:
            return 1
    
    p1_stocks = game.frames[-1].ports[port1].leader.post.stocks
    p2_stocks = game.frames[-1].ports[port2].leader.post.stocks
    
    if p1_stocks == p2_stocks:
        p1_damage = game.frames[-1].ports[port1].leader.post.damage
        p2_damage = game.frames[-1].ports[port2].leader.post.damage
        return p1_damage > p2_damage
    return int(p1_stocks > p2_stocks)


def is_valid_game(game):
    if game.metadata.duration < 15*60:
        return False
    if game.end.method.value != 3 and game.end.method.value != 1:
        return False
    if game.start.is_teams is True:
        return False
    if game.start.stage.value not in [2,3,8,28,31,32]:
        return False
    if len(get_ports(game))!=2:
        return False
    return True  
        

In [13]:
def make_rows(game, num_seconds, game_id):
    """
    Make a df for each game. Rows are timeslices, every num_seconds of the game, as well as the first and last frames
    
    """
    
    port1, port2 = get_ports(game)
    p1_char, p2_char = get_characters(game, port1, port2)
    stage = get_stage(game)
    winner = get_winner(game, port1, port2)
    duration = game.metadata.duration

    times = np.array(list(range(0,duration, num_seconds*60)) + [duration-1])
    num_times = len(times)
    p1_stocks, p2_stocks = np.zeros(num_times),np.zeros(num_times)
    p1_damage, p2_damage = np.zeros(num_times),np.zeros(num_times)
    
    for i, frame_num in enumerate(times):
        p1_stocks[i] = game.frames[frame_num].ports[port1].leader.post.stocks
        p2_stocks[i] = game.frames[frame_num].ports[port2].leader.post.stocks
        p1_damage[i] = game.frames[frame_num].ports[port1].leader.post.damage
        p2_damage[i] = game.frames[frame_num].ports[port2].leader.post.damage
    
    game_id_list = [game_id]*num_times
    p1_char_list = [p1_char]*num_times
    p2_char_list = [p2_char]*num_times
    stage_list = [stage]*num_times
    winner_list = [winner] * num_times
    return pd.DataFrame(data = {
            'id'     : game_id_list,
            'p1_char': p1_char_list,
            'p2_char': p2_char_list,
            'stage' : stage_list,
            'frames_elapsed'   : times,
            'p1_stocks': p1_stocks,
            'p2_stocks': p2_stocks,
            'p1_damage': p1_damage,
            'p2_damage': p2_damage,
            'winner' : winner_list
        })

In [14]:
def construct_df(file_list, num_seconds):
    df_list = []
    for slp_file in tqdm(file_list,desc='Games parsed'):
        try:
            game = Game(slp_file)
            if is_valid_game(game):
                game_id = slp_file.split('/')[-1][:-4]
                rows = make_rows(game, num_seconds,game_id)
                df_list.append(rows)
        except:
            continue
    return pd.concat(df_list).reset_index(drop=True)

def ohe_chars_stage(df):
    characters = ['FOX', 'FALCO', 'MARTH', 'SHEIK','JIGGLYPUFF', 'PEACH', 'ICE_CLIMBERS','CAPTAIN_FALCON',
                 'PIKACHU','SAMUS','DR_MARIO','YOSHI','LUIGI','GANONDORF','MARIO','YOUNG_LINK','DONKEY_KONG',
                 'LINK','GAME_AND_WATCH','ROY','MEWTWO','ZELDA','NESS','PICHU','BOWSER','KIRBY']

    stages = ['BATTLEFIELD', 'FINAL_DESTINATION',
               'DREAM_LAND_N64', 'YOSHIS_STORY', 'FOUNTAIN_OF_DREAMS','POKEMON_STADIUM']
    ohe = OneHotEncoder(categories=[characters,characters,stages], sparse=False)
    ohe_cols = ohe.fit_transform(df[['p1_char','p2_char','stage']])
    ohe_df = pd.DataFrame(ohe_cols, columns = ohe.get_feature_names(['p1','p2','stage']))
    return  pd.concat([df,ohe_df],axis=1)

    



In [119]:
fp9_5s = construct_df(fp9_files,5)

In [21]:
%%time
gs_5s = construct_df(gs_files, 5)

Games parsed: 100%|██████████| 3561/3561 [29:58<00:00,  1.98it/s]


CPU times: user 29min 8s, sys: 46 s, total: 29min 54s
Wall time: 29min 59s


In [158]:
fp9_5s_ohe = ohe_chars_stage(fp9_5s)

In [162]:
fp9_5s_ohe.to_pickle('saved_data/fp9_5s_ohe.pkl')

In [22]:
gs_5s_ohe = ohe_chars_stage(gs_5s)

In [25]:
gs_5s_ohe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103085 entries, 0 to 103084
Data columns (total 68 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   id                        103085 non-null  object 
 1   p1_char                   103085 non-null  object 
 2   p2_char                   103085 non-null  object 
 3   stage                     103085 non-null  object 
 4   frames_elapsed            103085 non-null  int64  
 5   p1_stocks                 103085 non-null  float64
 6   p2_stocks                 103085 non-null  float64
 7   p1_damage                 103085 non-null  float64
 8   p2_damage                 103085 non-null  float64
 9   winner                    103085 non-null  int64  
 10  p1_FOX                    103085 non-null  float64
 11  p1_FALCO                  103085 non-null  float64
 12  p1_MARTH                  103085 non-null  float64
 13  p1_SHEIK                  103085 non-null  f

In [24]:
gs_5s_ohe.to_pickle('saved_data/gs_5s_ohe.pkl')